In [10]:
import os
import pandas as pd
import zipfile
import csv
from io import StringIO
import mysql.connector

### Testing on home server

In [ ]:
try:
    connection = mysql.connector.connect(
        host = '192.168.1.26',
        user = 'aportra',
        password = '@$Occer22',
        database = 'wedge_assignment')

    cursor = connection.cursor()
except mysql.connector.Error as err:
    print(f'{err}')

finally:
    if connection.is_connected():
        connection.close()
        print('Connection Closed')
    
    if cursor:
        print('Cursor created')

#### Setting Directory for Mac and Windows 

In [12]:

try:
    directory = '/Users/aaronportra/Documents/Advanced Data Analytics/wedge_project/WedgeZipOfZips_small/'
    os.listdir(directory)
except FileNotFoundError:
    directory = 'C:\\Users\\aport\\OneDrive\\Documents\\School\\Applied Data Analytics\\wedge_project\\WedgeZipOfZips_small\\'
    os.listdir(directory)

## Cleaning Data for BQ

In [13]:
def sniff(file_path):
    with open(file_path, 'r', errors = 'replace') as f:
        sample = f.read(10000)

        sniffer = csv.Sniffer()

        f.seek(0)

        delimiter = sniffer.sniff(sample).delimiter

        f.seek(0)

        reader = csv.reader(f, delimiter=delimiter)

        first_row = next(reader)

        has_header = sniffer.has_header(sample)


        if len(first_row) < 50:
            has_header = False
        # if :

        #     has_header = False

    return delimiter, has_header

In [ ]:
sniff(directory + 'transArchive_201607_small.zip')

In [ ]:
sniff(directory + 'transArchive_201201_201203_inactive_small.zip')

In [16]:
transactions = {}

for file in os.listdir(directory):
    if file.endswith('.zip'):
        with zipfile.ZipFile(directory + file,'r') as zip_file:
            for info in zip_file.namelist():
                h = sniff(directory + file)

                file_content = zip_file.read(info).decode('utf-8')

                csv_file = StringIO(file_content)

                data = pd.read_csv(csv_file, delimiter = h[0])

                data = pd.DataFrame(data)


                if h[1]:

                    columns = data.columns

                if not h[1]:
                     
                     data.columns = columns
                
                transactions[file] = data

            
                

In [17]:
for col in transactions['transArchive_201609_small.zip'].columns:
    if col not in transactions['transArchive_201001_201003_small.zip'].columns:
        print(col)

In [ ]:
len(transactions['transArchive_201001_201003_small.zip'].columns)

In [19]:
for i,trans in enumerate(transactions):
   assert(len(transactions[trans].columns) == 50)


Should iterate through each to establish columns that contain NaN. Then can focus on how to handle each one. 

In [20]:
nan_cols = set()

for trans in transactions:
    for cols in transactions[trans]:
        if pd.isna(transactions[trans][cols])[0] == True:
            nan_cols.add(cols)

In [21]:
# for trans in transactions:
#     for cols in transactions[trans]:
#         if cols in nan_cols:
#             print(cols,' ', trans)

In [22]:
def retrieve_nans(dic):

    for trans in dic:
        for cols in dic[trans]:
            for value in dic[trans][cols].values:
                print(value)


            # for df in dic[trans][cols]:
            #     if df:
            #         print(df)
            #     # if dic[trans][cols].values.any == 'nan':
            #     #     print(df)

In [23]:
# retrieve_nans(transactions)

In [24]:
# type(transactions['transArchive_201001_201003_small.zip']['datetime'].values)